<a href="https://colab.research.google.com/github/vivacitylabs/data-toolkit/blob/master/notebooks/sensor_metadata_bulk_download_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sensor metadata - Bulk Download Generator



## Generate a csv file of sensor meta data

This notebook gives you access to VivaCity data via the API. It is an **interim solution** while we're working on dashboard developments.

You can contact customer support if you have any issues (support@vivacitylabs.com) or raise a ticked on the [Customer Help Portal](https://vivacitylabs.atlassian.net/servicedesk/customer/portal/16).

#### How it works

This notebook will run you through all the necessary steps and will output a csv file.

You just fill in a few details and hit the run button (▶) next to each code cell.

You will need your VivaCity **API login credentials**

#### Output format

You will receive a list of all your sensors (hardware) with their linked entities (lens, countlines, zones).

The API names match Dashboard terminology as follows

- Hardware Position (`hardware_pos`) = Sensor
- View_point = Lens
- Countline = Countline
- Zone = Zone

Please note that sensor numbers are derived from countline names and depend on consistency in countline names.


| derived_sensor_number | hardware_pos_id | hardware_pos_lat |  hardware_pos_lon |  view_point_id | entity |	entity_id | entity_geojson | zonal_speeds | dwell_times |turning_counts | occupancy |
|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|
| S5 |63502 |53.483353	| -2.25362 |	13427 | Countline | 52345 | {'type': 'LineString', 'coordinates': [[ ..	| FALSE	|  FALSE | FALSE	|  FALSE |
| S5 |63502 |53.483353	| -2.25362 |	13427 | Zone | 521 |	| FALSE	|  TRUE | FALSE	|  FALSE |

## Stage 1: Getting Started
Let's import the packages we need. Hit the run button (▶) in the top left.

In [1]:
#@title { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to import functions
import requests
import getpass
import json
import pandas as pd
from datetime import date, datetime, timedelta
import csv
import time
import pytz
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Stage 2: Request data

1. We authenticate the API user. If the user isn't setup properly, this will throw an error. Check you got the correct username and password.

2. We will get all the sensors available to the API user and clean data a bit.


ℹ  Sensor names are retrieved from countline names so they can sligthly differ if not named consistently.


### Authentication
Now you will need your API login details, ie. a username and a password. If you don't have one, please contact contact customer support (support@vivacitylabs.com).

1.   Enter the username into the field on the right, then hit the run button (▶).
2.   Input the password in the box that appears below it and hit "enter" on your keyboard.

In [2]:
#@title  {vertical-output: true, display-mode: "form" }
#@markdown #### Provide API credentials
#@markdown Insert your login credentials then hit run button
username = "api-username" #@param {type:"string"}

auth_body = {}
auth_body['username'] = username
auth_body['password'] = getpass.getpass()

··········


#### Get access token

In [3]:
#@title { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to get authorized access to the API
print("Authorising...")
auth_response = requests.post("https://api.vivacitylabs.com/get-token", data=auth_body, headers={'Content-Type':'application/x-www-form-urlencoded'})
if auth_response.status_code == 401:
  print("\n!Error: Can't connect to the API. Check your username and password again.\nIf issues persists, ask customer support if your user is setup correctly on the API\n")
else:
  headers = {}
  headers['Authorization'] = "Bearer " + auth_response.json()['access_token']
  refresh_body = {}
  refresh_body['refresh_token'] = auth_response.json()['refresh_token']
  start = time.time()
  print("Done. Successfully retrieved access token.")

Authorising...
Done. Successfully retrieved access token.


### Get sensor meta data

In [4]:
#@title { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to pull and process meta data
# API V3

#get hardware meta data
print("\nRequesting metadata ...")
api_url_base = 'https://beta.api.vivacitylabs.com'
hardware_request = requests.get(f'{api_url_base}/hardware/metadata', headers=headers)
if hardware_request.status_code == 401:
  print("\n!Error: Can't access the data. Check with customer support if your user is setup correctly on API 3\n")
hardware = hardware_request.json()

# Get hardware info
dict_hard = { "hardware_pos_id" : [], "hardware_pos_lat" : [], "hardware_pos_lon" : [], "view_point_id" : [],
             "entity":[], "entity_id": [], "entity_name": [] , "entity_geojson":[],
             "zonal_speed":[], "dwell_times":[], "turning_counts":[],"occupancy":[]}
for hwid in hardware:
  for vpid in hardware[hwid]["view_points"]:
    for countline_id in hardware[hwid]["view_points"][vpid]["countlines"]:
        dict_hard["hardware_pos_id"].append(hwid)
        dict_hard["hardware_pos_lat"].append(hardware[hwid]["lat"])
        dict_hard["hardware_pos_lon"].append(hardware[hwid]["long"])
        dict_hard["view_point_id"].append(vpid)
        dict_hard["entity"].append("Countline")
        dict_hard["entity_id"].append(countline_id)
        dict_hard["entity_name"].append(hardware[hwid]["view_points"][vpid]["countlines"][countline_id]['name'])
        dict_hard["entity_geojson"].append(hardware[hwid]["view_points"][vpid]["countlines"][countline_id]["geometry"]["geo_json"])
        dict_hard["zonal_speed"].append(False)
        dict_hard["dwell_times"].append(False)
        dict_hard["turning_counts"].append(False)
        dict_hard["occupancy"].append(False)
    for zone_id in hardware[hwid]["view_points"][vpid]["zones"]:
        dict_hard["hardware_pos_id"].append(hwid)
        dict_hard["hardware_pos_lat"].append(hardware[hwid]["lat"])
        dict_hard["hardware_pos_lon"].append(hardware[hwid]["long"])
        dict_hard["view_point_id"].append(vpid)
        dict_hard["entity"].append("Zone")
        dict_hard["entity_id"].append(zone_id)
        dict_hard["entity_name"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]['name'])
        dict_hard["entity_geojson"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]["geometry"]["geo_json"])
        dict_hard["zonal_speed"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]['is_speed'])
        dict_hard["dwell_times"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]['is_dwell_times_zone'])
        dict_hard["turning_counts"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]['is_turning'])
        dict_hard["occupancy"].append(hardware[hwid]["view_points"][vpid]["zones"][zone_id]['is_occupancy'])

# convert to data frame
df = pd.DataFrame.from_dict(dict_hard)

# derive sensor number from countline names
sensor_name = df[df["entity"]=="Countline"].copy()
sensor_name['derived_sensor_number'] = sensor_name['entity_name'].str.extract('([S,s]_?\d{1,3})', expand=True)
sensor_name = sensor_name[~sensor_name["entity_geojson"].isna()] # remove countlines with no geometry data
sensor_name = sensor_name.drop_duplicates(subset='hardware_pos_id', keep='first').sort_values(by='derived_sensor_number',ascending=True).reset_index(drop=True)

# merge derived sensor names
df = pd.merge(df, sensor_name[["hardware_pos_id", "derived_sensor_number"]], left_on="hardware_pos_id", right_on="hardware_pos_id", how="left")
df["derived_sensor_number"] = df["derived_sensor_number"].fillna("Unknown")

# reorder columns
df = df[['derived_sensor_number', 'hardware_pos_id', 'hardware_pos_lat', 'hardware_pos_lon',
       'view_point_id', 'entity', 'entity_id', 'entity_name', 'entity_geojson',
       'zonal_speed', 'dwell_times', 'turning_counts', 'occupancy']]

print("done")


Requesting metadata ...
done


## Stage 3: Data Export
Now let's write this to a .csv file. You can either save the file locally (it will show in your Downloads folder) or save it to a Google Drive.

* **Local Downloads Folder:** This might not work if your browser or computer blocking downloads.
* **Google Drive:** If you want to save it in Google Drive, you will be asked for permission to connect to your Google Account.

In [33]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown Select where to save the csv file
download_location = "Local folder" #@param [ "Local folder", "Google Drive"]
#@markdown Name your file
filename = "sensor_metadata" #@param {type:"string"}
#@markdown Hit run (>)
df_export = df

if download_location == "Local folder":
  from google.colab import files
  df_export.to_csv(filename + ".csv", index = False)
  files.download(filename + ".csv")
else:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/'
  df_export.to_csv(path + filename +".csv", index = False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>